In [1]:
from google.colab import drive
drive.mount('/content/drive')
import sys
sys.path.append("/content/drive/MyDrive/second_year_semester_B/Cognitive_of_Animal/collective-motion")

Mounted at /content/drive


In [2]:
from tools import *
import torch

<div dir="rtl" lang="he" xml:lang="he">

# תנועה קולקטיבית - מודלים טובים ומודלים טובים יותר


תנועה קולקטיבית היא אחד התופעות המענייונות ביותר,
קבוצה של בעלי חיים נעה במרחב בצורה כאילו מאורגנת ללא שוב מנהיג נראה לעין.
אלו מודלים יכולים להסביר את התנועה הזאת?
והאם נמצא דרך להשוות בן המודלים האלו?

<div dir="rtl" lang="he" xml:lang="he">

## מודל פשוט - vicsek
המודל הבסיסי ביותר, הוא המודל של
vicsek
כל בעל חיים מתואר ע"י מיקומו,
מסומן ב
$\vec{r}_i \in R^3$
ווקטור הכיוון שלו שמסומן ב
$\vec{v}_i \in R^2$
כאשר כלל העידכון הוא הליכה בכיון הממוצע.
ע"י חישוב וקטור הכיון של השכנים, מתואר ע"י המשוואה
$$
\\ \vec{S}_i = \sum_{j\in \text{Neg}(i)}{\vec{v}_j}
\\ \vec{v}_i = \frac{\vec{S}_i}{|\vec{S}_i|}*v_0
$$
כאשר
$v_0$
הוא גודל קבוע המייצג את מהירות בעל החיים.




$$
\\ \vec{S} = \sum_{j\in \text{Neg}(i)}{\vec{v}_j}
\\ \vec{v}_i = \frac{\vec{S}}{|\vec{S}|}*v_0
$$

In [3]:
def get_avrage_velocity(data: TensorType, radios: float = 5.0) -> TensorType:
    """get the avrage velocity of the input data

    Args:
        data (TensorType): tensor (N,5) of x,y,z,theta,phi
        radios (float, optional): radios of . Defaults to 5.0.

    Returns:
        TensorType: tensor (N,2) new velocity of the input data
    """
    # for each data point, get the distance from any other data point
    dis_mat = torch.stack(
        [torch.linalg.norm(data[i, :3] - data[:, :3], dim=1) for i in range(len(data))]
    )
    # dis_mat = dis_mat + dis_mat.T
    # return new_data
    # for each data point, get the avrage velocity of the data points that are in the radios
    new_velocity = torch.stack(
        [data[dis_mat[i] < radios, 3:].mean(dim=0) for i in range(len(data))]
    )
    return new_velocity

In [6]:
device= "cuda" if torch.cuda.is_available() else "cpu"

In [10]:
def f_cuda():
  data = torch.rand(1000, 5).to("cuda")
  data[:, 3:] = data[:, 3:] * 360
  data[:, :3] = data[:, :3] * 20 - 10
  get_avrage_velocity(data)
def f_cpu():
  data = torch.rand(1000, 5).to("cpu")
  data[:, 3:] = data[:, 3:] * 360
  data[:, :3] = data[:, :3] * 20 - 10
  get_avrage_velocity(data)



In [11]:
import timeit
print(timeit.timeit("f_cuda()",number=100,globals=globals()))
print(timeit.timeit("f_cpu()",number=100,globals=globals()))

12.468634613999996
7.482020849000037


In [ ]:
torch.tensor([i for i in range(10)])

tensor([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [ ]:
for r in torch.logspace(0, 10**7,10):
    print(
        f"list comp = {timeit.timeit(torch.tensor([i for i in range(int(r))]), number=100)}"
    )
    print(f"torch range = {timeit.timeit(torch.range(int(r)), number=100)}")

ValueError: stmt is neither a string nor callable